# 1. Import Library

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# 2. Gpu or Cpu

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPU 사용 가능 여부 확인
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")

# 3. Hyperparameter

In [ ]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

# 4. Load Data

In [ ]:
# MNIST dataset을 가져오고 transforms를 통해 이를 Tensor 객체로 가공. Train set과 Test set으로 구성
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

# DataLoader를 통해 dataset의 전체 데이터를 batch size로 slice해 실제로 학습할 때 이용할 수 있는 형태로 생성
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

# 5. Model Define

In [ ]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # in_channels: 1, out_channels: 10, kernel_size: 5, stride: 1, padding: 0인 convolution 연산
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # in_channels: 10, out_channels: 20, kernel_size: 5, stride: 1, padding: 0인 convolution 연산
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # p: 0.25로 학습시 dropout 진행, inplace: False
        self.mp = nn.MaxPool2d(2) # kernel_size=2로 Max pool 진행
        self.fc1 = nn.Linear(320,100) # in_features: 320, out_features: 100
        self.fc2 = nn.Linear(100,10) # in_features: 100, out_features: 10 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) #relu 연산
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) # flatten
        x = self.fc1(x) 
        x = self.fc2(x) 
        return F.log_softmax(x) 

# 6. Train

In [ ]:
model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device) # CrossEntropyLoss을 연산
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # Adam optimizer 사용
 
for epoch in range(epochs): 
    avg_cost = 0 #초기화

    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() #모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : backward()를 통해 구한 gradient가 누적됨
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward() #gradient 계산
        optimizer.step() #파라미터 갱신
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# 7. Evaluate



In [ ]:
model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] # 가장 큰 값을 예측값으로 설정
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')